In [56]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

In [57]:
import hvplot.pandas

DATA PROCESSING

In [58]:
#df=pd.read_csv('C:\\Users\\aniru\\Desktop\\co2_dashboard\\data.csv')
df=pd.read_csv('data.csv')

In [59]:
df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.157,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.156,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.156,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.155,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47410,Zimbabwe,2018,ZWE,15052191.0,2.678627e+10,0.558,0.037,10.715,1.419,15.265,...,NaN,0.114,0.001,0.001,0.002,0.0,116.76,29.37,-0.088,-0.825
47411,Zimbabwe,2019,ZWE,15354606.0,2.514642e+10,0.473,0.031,9.775,-0.939,-8.765,...,NaN,0.113,0.001,0.001,0.002,0.0,116.03,28.70,0.143,1.463
47412,Zimbabwe,2020,ZWE,15669663.0,2.317871e+10,0.496,0.032,7.850,-1.926,-19.700,...,NaN,0.112,0.001,0.001,0.002,0.0,113.20,25.99,0.818,10.421
47413,Zimbabwe,2021,ZWE,15993525.0,2.514009e+10,0.531,0.033,8.396,0.547,6.962,...,NaN,0.110,0.001,0.001,0.002,0.0,NaN,NaN,1.088,12.956


In [60]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [61]:
df=df.fillna(0)
df['gdp_per_capita']=np.where(df['population']!=0,df['gdp']/df['population'],0)

In [62]:
idf=df.interactive()

co2 emission over time continent

In [63]:
year_slider=pn.widgets.IntSlider(name='year slider',start=1750,end=2020,step=5,value=1850)
year_slider

IntSlider(end=2020, name='year slider', start=1750, step=5, value=1850)

In [64]:
#RADIO BUTTON FOR CO2 MEASURES
yaxis_co2= pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['co2','co2_per_capita',],
    button_type='success'
)
        

In [65]:
continents=['World','Asia','Oceania','Europe','Africa','North America','South America','Antarctica']

co2_pipeline=(
    idf[
    (idf.year<=year_slider)&
    (idf.country.isin(continents))
    ]
    .groupby(['country','year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)


In [66]:
co2_pipeline

In [67]:
co2_plot=co2_pipeline.hvplot(x='year',by='country',y=yaxis_co2,line_width=2,title="CO2 emission by continents")

In [68]:
co2_plot

TABLE- CO2 emission over time by continent

In [69]:
co2_table=co2_pipeline.pipe(pn.widgets.Tabulator,pagination='remote',page_size=10,sizing_mode='stretch_width')

In [70]:
co2_table

CO2 vs GDP scatterplot

In [71]:
co2_vs_gdp_scatterplot_pipeline =(
    idf[
     (idf.year==year_slider)&
    (~(idf.country.isin(continents)))
    ]
    .groupby(['country','year','gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [72]:
co2_vs_gdp_scatterplot_pipeline

In [73]:
co2_vs_gdp_scatterplot=co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita',y='co2',size=80,
                                                              kind='scatter',alpha=0.7,legend=False,height=500,width=500)

In [74]:
co2_vs_gdp_scatterplot

CO2 source by continent

In [82]:
yaxis_co2_source=pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['coal_co2','oil_co2','gas_co2'],
    button_type='success'
)
continents_excl_world=['Asia','Oceania','Europe','Africa','North America','South America','Antarctica']
c02_source_bar_pipeline=(
    idf[
    (idf.year==year_slider)&
    (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year','country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [83]:
co2_source_bar_plot=c02_source_bar_pipeline.hvplot(kind='bar',x='country',y=yaxis_co2_source,title='CO2 source by continent')
co2_source_bar_plot

CREATING dashboard

In [85]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();